# ==================== 1. Install ====================

In [1]:

!pip install transformers datasets sentence-transformers faiss-cpu streamlit pyngrok --quiet


# ==================== 2. Imports ====================

In [2]:

import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


# ==================== 3. Load Emotion Dataset ====================

In [3]:

dataset = load_dataset("tweet_eval", "emotion")

# نجهز train/test (ناخد عينة صغيرة للتجربة)
train_data = dataset["train"].shuffle(seed=42).select(range(500))
test_data = dataset["test"].shuffle(seed=42).select(range(10))  # 10 أسئلة للتست


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# ==================== 4. Load Emotion Detection Model ====================

In [4]:
model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
classifier = pipeline("text-classification", model=model_name, return_all_scores=True)


Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


# ==================== 5. Build RAG Corpus ====================

In [5]:
corpus = {
    "sadness": [
        "I'm really sorry you're feeling this way. Want to talk about it?",
        "It's okay to feel sad sometimes, I'm here for you.",
        "I hear you. That sounds really tough."
    ],
    "joy": [
        "That's wonderful! What made you so happy?",
        "I’m glad to hear that! Keep smiling!",
        "That’s amazing! Tell me more about it."
    ],
    "anger": [
        "I can sense your frustration. Want to share more?",
        "It's normal to feel angry sometimes. What triggered it?",
        "Take a deep breath, I’m listening."
    ],
    "fear": [
        "It’s okay to feel scared, you’re not alone.",
        "I understand, that must feel overwhelming.",
        "Want to talk about what’s worrying you?"
    ],
    "love": [
        "That’s beautiful! Love is such a strong emotion.",
        "It sounds like you care deeply. That’s wonderful.",
        "Love can be so powerful. Tell me more!"
    ],
    "surprise": [
        "Wow, that sounds surprising indeed!",
        "Really? That must have been unexpected!",
        "That’s quite something! How did you feel about it?"
    ]
}

# Convert corpus to documents list
documents = []
labels = []
for emotion, responses in corpus.items():
    for resp in responses:
        documents.append(resp)
        labels.append(emotion)


# ==================== 6. Embedding + FAISS ====================

In [6]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(documents)

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

def retrieve_response(emotion, query, k=3):
    query_emb = embedder.encode([query])
    distances, indices = index.search(query_emb, k)
    results = [documents[i] for i in indices[0] if labels[i] == emotion]
    return results if results else [documents[indices[0][0]]]


# ==================== 7. Few-Shot Prompt ====================


In [7]:

few_shot_examples = [
    {"q": "I'm so sad today", "a": "I'm sorry you're going through this. Want to share more?"},
    {"q": "I feel really happy!", "a": "That's amazing! What’s bringing you joy?"},
    {"q": "I'm so angry right now", "a": "I understand, anger is tough. Want to talk about what happened?"}
]

def build_prompt(user_msg, emotion, retrieved):
    prompt = "The user is feeling " + emotion + ". Respond empathetically.\n\n"
    for ex in few_shot_examples:
        prompt += f"User: {ex['q']}\nBot: {ex['a']}\n\n"
    prompt += f"User: {user_msg}\nBot: {retrieved[0]}"
    return prompt


# ==================== 8. Streamlit UI ====================


In [10]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss, numpy as np

# Re-load lightweight components inside Streamlit
emotion_model = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", return_all_scores=True)
embedder = SentenceTransformer("all-MiniLM-L6-v2")



corpus = {
    "sadness": [
        "I'm really sorry you're feeling this way. Want to talk about it?",
        "It's okay to feel sad sometimes, I'm here for you.",
        "I hear you. That sounds really tough.",
        "I understand, it must be heavy on you.",
        "Sending you strength, you’re not alone in this.",
        "Your feelings are valid. Take your time to heal.",
        "I’m here to listen whenever you want to share more."
    ],
    "joy": [
        "That's wonderful! What made you so happy?",
        "I’m glad to hear that! Keep smiling!",
        "That’s amazing! Tell me more about it.",
        "I love hearing good news from you!",
        "It sounds like you’re really enjoying yourself!",
        "That’s fantastic! I’m happy for you.",
        "Joy like this is contagious—thank you for sharing!"
    ],
    "anger": [
        "I can sense your frustration. Want to share more?",
        "It's normal to feel angry sometimes. What triggered it?",
        "Take a deep breath, I’m listening.",
        "I understand, anger can be overwhelming.",
        "It’s okay to vent, I’m here to hear you out.",
        "I hear your frustration—want to talk about a solution?",
        "It must be tough dealing with this, I get it."
    ],
    "fear": [
        "It’s okay to feel scared, you’re not alone.",
        "I understand, that must feel overwhelming.",
        "Want to talk about what’s worrying you?",
        "Sometimes fear can feel paralyzing, I hear you.",
        "You’re safe here, take a breath.",
        "I know it’s not easy, but you’re stronger than you think.",
        "It’s completely natural to feel this way."
    ],
    "love": [
        "That’s beautiful! Love is such a strong emotion.",
        "It sounds like you care deeply. That’s wonderful.",
        "Love can be so powerful. Tell me more!",
        "That’s sweet—love makes life brighter.",
        "I can feel the warmth in your words.",
        "Cherishing love is one of the best things we can do.",
        "That’s amazing, love truly brings joy."
    ],
    "surprise": [
        "Wow, that sounds surprising indeed!",
        "Really? That must have been unexpected!",
        "That’s quite something! How did you feel about it?",
        "I wasn’t expecting that either—amazing!",
        "Life can be full of surprises, right?",
        "That must have caught you off guard.",
        "Sounds exciting! What happened next?"
    ],
    "disgust": [
        "That sounds really unpleasant, I understand your reaction.",
        "I can see why you’d feel disgusted.",
        "It must have been uncomfortable to deal with.",
        "That reaction is completely natural.",
        "I hear you, it doesn’t sound nice at all.",
        "Your feelings are valid—it’s okay to feel this way.",
        "That situation would upset anyone."
    ],
    "optimism": [
        "I love your positive outlook! Keep it up.",
        "That’s a great way to see things.",
        "Staying hopeful can make a big difference.",
        "I admire your optimism—it’s inspiring.",
        "That mindset will help you move forward.",
        "It’s wonderful that you can see the bright side.",
        "Keep believing in yourself!"
    ],
    "neutral": [
        "Thanks for sharing that with me.",
        "I hear you—tell me more.",
        "That’s interesting, I’d like to understand better.",
        "I appreciate you opening up.",
        "I’m listening—want to go deeper?",
        "Okay, I get what you’re saying.",
        "That makes sense, thanks for explaining."
    ]
}

documents = []
labels = []
for emotion, responses in corpus.items():
    for resp in responses:
        documents.append(resp)
        labels.append(emotion)

doc_embeddings = embedder.encode(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

def retrieve_response(emotion, query, k=3):
    query_emb = embedder.encode([query])
    distances, indices = index.search(query_emb, k)
    results = [documents[i] for i in indices[0] if labels[i] == emotion]
    return results if results else [documents[indices[0][0]]]

st.set_page_config(page_title="EmpathyBot 💙")
st.title("EmpathyBot: Sentiment-Driven Chat 🤖")

if "history" not in st.session_state:
    st.session_state.history = []

user_input = st.text_input("💬 Your message:")

if st.button("Send"):
    if user_input.strip():
        emotion_pred = emotion_model(user_input)
        top_emotion = max(emotion_pred[0], key=lambda x: x["score"])["label"]
        retrieved = retrieve_response(top_emotion, user_input)
        bot_answer = retrieved[0] + " (Disclaimer: I’m not a therapist.)"
        st.session_state.history.append(("User", user_input))
        st.session_state.history.append(("Bot", bot_answer))

for speaker, msg in st.session_state.history:
    if speaker == "User":
        st.markdown(f"**🧑 You:** {msg}")
    else:
        st.markdown(f"**🤖 Bot:** {msg}")


Overwriting app.py


# ==================== 10. Run Streamlit with Ngrok ====================

In [11]:
# ==================== 10. Run Streamlit with Ngrok ====================
!ngrok config add-authtoken 32QbKiExvplbbxkT3iJv6kvUElc_5vEnyioVbkr2HUk27C3Sw   # ضع التوكن بتاع ngrok هنا
!streamlit run app.py --server.port 7860 &>/dev/null&

from pyngrok import ngrok
ngrok.kill()  # اقفل كل الأنفاق المفتوحة
public_url = ngrok.connect(7860)
print("🔥 Streamlit URL:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔥 Streamlit URL: NgrokTunnel: "https://9188986aab0e.ngrok-free.app" -> "http://localhost:7860"
